In [1]:
import pandas as pd

input_ = pd.read_excel('./5.vehicle_1.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_

,No,Item Number,Market,Page,JOIN_INP,Row,Year,Make,Model,Trim,Engine,Notes
0,1,295813156245,US,1,295813156245;1,1,2009,Audi,A8 Quattro,Base Sedan 4-Door,4.2L 4163CC V8 GAS DOHC Naturally Aspirated,A8 D3
1,1,295813156245,US,1,295813156245;1,2,2009,Audi,A8 Quattro,L Sedan 4-Door,4.2L 4163CC V8 GAS DOHC Naturally Aspirated,A8 D3
2,1,295813156245,US,1,295813156245;1,3,2009,Audi,A8 Quattro,L Sedan 4-Door,4.2L 4172CC V8 GAS DOHC Naturally Aspirated,A8 D3
3,1,295813156245,US,1,295813156245;1,4,2009,Audi,A8 Quattro,L Sedan 4-Door,6.0L 5998CC 366Cu. In. W12 GAS DOHC Naturally ...,A8 D3
4,1,295813156245,US,1,295813156245;1,5,2009,Audi,A8 Quattro,Plus Sedan 4-Door,4.2L 4172CC V8 GAS DOHC Naturally Aspirated,A8 D3
...,...,...,...,...,...,...,...,...,...,...,...,...
81424,792,295541445177,US,2,295541445177;2,17,2007,Toyota,Yaris,Premium Hatchback 4-Door,1.5L 1497CC l4 GAS DOHC Naturally Aspirated,
81425,792,295541445177,US,2,295541445177;2,18,2007,Toyota,Yaris,Premium Sedan 4-Door,1.5L 1497CC l4 GAS DOHC Naturally Aspirated,
81426,792,295541445177,US,2,295541445177;2,19,2007,Toyota,Yaris,RS Hatchback 2-Door,1.5L 1497CC l4 GAS DOHC Naturally Aspirated,
81427,792,295541445177,US,2,295541445177;2,20,2007,Toyota,Yaris,RS Hatchback 4-Door,1.5L 1497CC l4 GAS DOHC Naturally Aspirated,


In [2]:
input_2 = input_[['Item Number']].drop_duplicates(keep='first', ignore_index=True)
input_2 = input_2.sort_values(by=['Item Number'],
                              ascending=[True],
                              ignore_index=True)

input_2

,Item Number
0,272480727913
1,272647174817
2,272647351668
3,272648503165
4,272779080668
...,...
785,296544428214
786,296544489937
787,296561560566
788,296561563360


In [3]:
from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    df_temp = input_[input_['Item Number'] == input_2.loc[i, 'Item Number']].copy().reset_index(drop=True)

    # = = = = = = = = = = = = = = =

    dict_vehicle = {}
    for j in range(len(df_temp)):
        if df_temp.loc[j, 'Market'] in ['US', 'UK', 'AU']:
            make, model, engine, year, year_2 = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Engine'], int(df_temp.loc[j, 'Year']), int(df_temp.loc[j, 'Year'])
        elif df_temp.loc[j, 'Market'] == 'DE':
            make, model, engine, year, year_2 = df_temp.loc[j, 'Marke'], df_temp.loc[j, 'Modell'], df_temp.loc[j, 'Motor'], int(df_temp.loc[j, 'Baujahr'].split('-')[0].split('/')[0]), int(df_temp.loc[j, 'Baujahr'].split('-')[1].split('/')[0])

        if make not in dict_vehicle.keys():
            dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
        elif model not in dict_vehicle[make].keys():
            dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
        elif engine not in dict_vehicle[make][model].keys():
            dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
        else:
            if year not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year)
            if year_2 not in dict_vehicle[make][model][engine]:
                dict_vehicle[make][model][engine].append(year_2)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            list_year = []
            for engine in dict_vehicle[make][model].keys():
                list_year = list(set(list_year + dict_vehicle[make][model][engine]))
            year_begin = min(list_year)
            year_end = max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(sorted(list_vehicle))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make in dict_vehicle.keys():
        for model in dict_vehicle[make].keys():
            for engine in dict_vehicle[make][model].keys():
                year_begin = min(dict_vehicle[make][model][engine])
                year_end = max(dict_vehicle[make][model][engine])
                list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

    input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(sorted(list_vehicle))

# = = = = = = = = = = = = = = =

input_2.to_excel('./6.vehicle.xlsx', index=False)

input_2

Progress: 100%|███████████████████████████| 790/790 [00:06<00:00, 118.49it/s]


,Item Number,Vehicle W/o Engine,Vehicle W/ Engine
0,272480727913,Chrysler Town & Country 2001-2007\nChrysler Vo...,Chrysler Town & Country 2001-2003 3.3L 3301CC ...
1,272647174817,Dodge Dakota 1996-2001\nDodge Durango 1998-200...,Dodge Dakota 1996 5.2L 5211CC 318Cu. In. V8 CN...
2,272647351668,Hyundai Santa Fe 2001-2006\nHyundai Sonata 199...,Hyundai Santa Fe 2001-2006 2.4L 2351CC l4 GAS ...
3,272648503165,Nissan Maxima 2008-2014\nNissan Murano 2009-20...,Nissan Maxima 2008-2014 3.5L 3498CC V6 GAS DOH...
4,272779080668,Suzuki Esteem 1995-2000\nSuzuki Grand Vitara 1...,Suzuki Esteem 1995-2000 1.6L 1590CC 97Cu. In. ...
...,...,...,...
785,296544428214,Dodge Dakota 2002-2003\nDodge Durango 2002-200...,Dodge Dakota 2002-2003 4.7L 287Cu. In. V8 GAS ...
786,296544489937,Honda Civic 2001-2005\nHonda Prelude 1997-2001,Honda Civic 2001-2005 1.7L 1668CC l4 CNG SOHC ...
787,296561560566,Chevrolet Colorado 2017-2022\nGMC Canyon 2017-...,Chevrolet Colorado 2017-2022 3.6L 222Cu. In. V...
788,296561563360,Nissan Maxima 2015-2020\nNissan Murano 2015-2020,Nissan Maxima 2015-2020 3.5L 3498CC V6 GAS DOH...
